[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shhommychon/detect-and-distort-faces/blob/main/run.ipynb)

# 환경설정

## [ngrok](https://ngrok.com/) 세팅

In [1]:
# NGROK_TOKEN = "hello_hommy"
# PASSWORD = "hello_hommy"

# !pip install colab_ssh --upgrade

# from colab_ssh import launch_ssh
# launch_ssh(NGROK_TOKEN, PASSWORD)

we highly recommend that update your code by following this documentation https://github.com/WassimBenzarti/colab-ssh#getting-started
Successfully running -1.tcp.ngrok.io:0
[Optional] You can also connect with VSCode SSH Remote extension using this configuration:

  Host google_colab_ssh
    HostName -1.tcp.ngrok.io
    User root
    Port 0
    


## colab에 필요 파일 배치

In [2]:
!git clone https://github.com/shhommychon/detect-and-distort-faces.git

import sys
sys.path.append("detect-and-distort-faces")

Cloning into 'detect-and-distort-faces'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 15 (delta 1), reused 15 (delta 1), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [3]:
!pip -qq install -r detect-and-distort-faces/requirements.txt

import os
os.mkdir("/root/.deepface")
os.mkdir("/root/.deepface/weights")

     |████████████████████████████████| 65 kB 2.2 MB/s 
     |████████████████████████████████| 32.8 MB 241 kB/s 
     |████████████████████████████████| 60.5 MB 1.2 MB/s 
     |████████████████████████████████| 2.3 MB 34.8 MB/s 
     |████████████████████████████████| 87 kB 6.7 MB/s 
     |████████████████████████████████| 462 kB 46.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


# 얼굴 탐지

In [4]:
SCREEN_WIDTH = 640; SCREEN_HEIGHT = 480

face_detect = False
face_detect_model = "none"
face_distort_type = "none"

In [ ]:
from detect_faces.deepface import deepface_detector
from detect_faces.opencv import haar_cascade_classifier
from helper import google_colab as colab

import cv2
import numpy as np

# start streaming video from webcam
colab.video_stream(width=SCREEN_WIDTH, height=SCREEN_HEIGHT)

# label for video
label_html = "Capturing..."

# initialze bounding box to empty
bbox = ''

while True:
    js_reply = colab.video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = colab.js_to_image(js_reply["img"])

    if face_detect:
        # 아래 옵션 중 얼굴 탐지 모델을 선택하세요.
        if face_detect_model == "cv2_haar":
            faces = haar_cascade_classifier(img)
        elif face_detect_model == "df_cv":
            faces = deepface_detector(img, detector_backend="opencv")
        elif face_detect_model == "df_ssd":
            faces = deepface_detector(img, detector_backend="ssd")
        elif face_detect_model == "df_dlib":
            faces = deepface_detector(img, detector_backend="dlib")
        elif face_detect_model == "df_mc":
            faces = deepface_detector(img, detector_backend="mtcnn")
        elif face_detect_model == "df_ret":
            faces = deepface_detector(img, detector_backend="retinaface")
        elif face_detect_model == "df_mp":
            faces = deepface_detector(img, detector_backend="mediapipe")
        else:
            faces = []

        bbox_array = np.zeros([SCREEN_HEIGHT, SCREEN_WIDTH, 4], dtype=np.uint8)
        
        for x, y, w, h in faces:
            bbox_array = cv2.rectangle(bbox_array, (x,y), (x+w,y+h), (255,0,0), 2) # draw rectangle to main image
            bbox_array[:,:,3] = (bbox_array.max(axis=2) > 0).astype(int) * 255

            bbox_bytes = colab.bbox_to_bytes(bbox_array)
            bbox = bbox_bytes
